# Sparse Linear Algebra 

Introduction and tutorial

In [2]:
import numpy as np
from scipy.sparse import csr_matrix

In [72]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py

Installed autotime.py. To use it, type:
  %load_ext autotime
time: 1.88 s


In [73]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.01 ms


## Introduction

Motivation: sparse matrices are very common in NLP.

The basic idea is that in sparse matrices most values are zero so we only keep track of the non-zero elements (data). We do this by specifing the indices of where these non-zero data live.

Compressed Sparse Row Matrices:

can create via either of the following:
1. numpy matrix
2. i,j and data
3. index pointers and data

index pointers:
* indptr[i] stores the index in values where row i starts
* indices[indptr[i]:indptr[i+1] are the column indices for the non-zeroes in row i

concerning the index pointer:

* indices is array of column indices
* data is array of corresponding nonzero values
* indptr points to row starts in indices and data
* length is n_row + 1, last item = number of values = length of both indices and data
* nonzero values of the i-th row are data[indptr[i]:indptr[i+1]] with column indices indices[indptr[i]:indptr[i+1]]
* item (i, j) can be accessed as data[indptr[i]+k], where k is position of j in indices[indptr[i]:indptr[i+1]]



various types of sparse matrices:
http://docs.scipy.org/doc/scipy/reference/sparse.html


### 1. create via numpy 

In [28]:
A = np.array([[1, 0],
       [2, 3],
       [4, 5]])

In [30]:
As = csr_matrix(A)
print As.toarray()

[[1 0]
 [2 3]
 [4 5]]


In [31]:
print As

  (0, 0)	1
  (1, 0)	2
  (1, 1)	3
  (2, 0)	4
  (2, 1)	5


###  2. Create CSR by using data and (i,j)

In [32]:
row = np.array([0,  1, 1, 2, 2])
col = np.array([0,  0, 1, 0, 1])
data = np.array([1,  2, 3, 4, 5])

In [33]:
As = csr_matrix((data, (row, col)), shape=(3, 2))
A = As.toarray()
A

array([[1, 0],
       [2, 3],
       [4, 5]])

In [34]:
print As

  (0, 0)	1
  (1, 0)	2
  (1, 1)	3
  (2, 0)	4
  (2, 1)	5


###  3.  Create csr matrix using index pointers

### Setup Some Matrices

In [57]:
def create_sparse_random_matrix(m,n,s):
    """ returns a sparse numpy matrix
    m, n define dense matrix dimensions
    define create sparse values only, via s
    """
    rows = np.array([np.random.randint(s) for i in range(s)])
    cols = np.array([np.random.randint(s) for i in range(s)])
    data = np.array([np.random.rand() for i in range(s)])
    return csr_matrix((data, (rows,cols)), shape=(m,n))

In [130]:
m = 10000
n = 100
s = 10

time: 1.2 ms


In [131]:
As = create_sparse_random_matrix(m,n,s)
print As

  (1, 8)	0.220724648672
  (2, 7)	0.50063224403
  (5, 9)	0.834911447733
  (6, 2)	0.195659312118
  (7, 3)	0.421571957583
  (7, 5)	0.357564640626
  (8, 9)	0.930878151319
  (9, 3)	0.794779911299
  (9, 8)	0.746377359636
  (9, 9)	0.211635398616
time: 2.57 ms


In [132]:
Bs = create_sparse_random_matrix(m,n,s)
print Bs

  (0, 0)	0.177527077285
  (3, 8)	0.484026089614
  (4, 2)	0.285934115737
  (4, 4)	0.876117102296
  (5, 9)	0.428277748541
  (7, 3)	1.04462506779
  (8, 1)	0.357292699143
  (9, 0)	0.63677594768
time: 3 ms


In [142]:
bs = csr_matrix(np.array([ np.random.rand()*np.random.randint(0,2) for i in range(m)]).reshape(m,1))
bs2 = csr_matrix(np.array([ np.random.rand()*np.random.randint(0,2) for i in range(m)]).reshape(m,1))

time: 41.1 ms


In [143]:
b = bs.toarray()
b2 = bs2.toarray()

time: 2.12 ms


In [135]:
b.shape, bs.shape, len(bs.data)

((10000, 1), (10000, 1), 5020)

time: 3.31 ms


In [144]:
#print bs

time: 708 µs


## Matrix-matrix contractions

In [137]:
# produce m,m matrx
Cs = As.dot(Bs.T)

time: 1.99 ms


In [138]:
A = As.todense()
B = Bs.todense()

time: 5.64 ms


In [140]:
C = A.dot(B.T)

time: 875 ms


## Matrix-vector contractions

In [139]:
Ds = Cs.dot(b)

time: 1.22 ms


In [141]:
D = C.dot(b)

time: 41.6 ms


## Vector-vector contractions

In [145]:
bs.shape

(10000, 1)

time: 2.78 ms


In [149]:
(bs.T).dot(bs2)

<1x1 sparse matrix of type '<type 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Column format>

time: 3.54 ms


In [150]:
(b.T).dot(b2)

array([[ 619.36503416]])

time: 3.31 ms


## Matrix additions

In [151]:
b2 = b - 0.5* b

time: 1 ms


In [152]:
bs2 = bs - 0.5*bs

time: 1.85 ms


In [153]:
sum(bs2)

<1x1 sparse matrix of type '<type 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

time: 1.59 s


In [154]:
print sum(bs2)

  (0, 0)	1238.07731773
time: 1.62 s


In [155]:
sum(b2)

array([ 1238.07731773])

time: 9.29 ms


## Solve linear systems

In [120]:
from scipy.sparse.linalg import spsolve
#from numpy.linalg import solve
from scipy import linalg

time: 1.21 ms


In [116]:
d = np.random.rand(m)
d.shape

(1000,)

time: 3.04 ms


In [117]:
xs = spsolve(Cs, d)

time: 1.3 ms


In [126]:
#x = np.linalg.solve(C, b)
#x = linalg.solve(C, b)

time: 777 µs


In [ ]:
#error = stats.norm(xs-x)
#error

## Decompositions

scipy sparse svds:

http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html

there are two issues here, one is time and the other is memory consumption. The latter is not reflected in this notebook, but svds should perform better.


eg see this post:

http://fa.bianp.net/blog/2012/singular-value-decomposition-in-scipy/

## scipy svds

In [127]:
from scipy.sparse.linalg import svds

time: 915 µs


In [234]:
u, ss, vt = svds(As, 99, which = 'LM')

time: 520 ms


In [235]:
type(u), type(ss), type(vt)

(numpy.ndarray, numpy.ndarray, numpy.ndarray)

time: 2.78 ms


In [236]:
u.shape, ss.shape, vt.shape

((10000, 99), (99,), (99, 100))

time: 2.84 ms


In [237]:
ss

array([ 0.11959854,  0.19565931,  0.45557503,  0.50063224,  1.09273719,
        1.31739244,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

time: 4.66 ms


In [238]:
largest_s = np.argmax(ss)

time: 1.08 ms


In [233]:
sum(np.abs(u[:,largest_s])), sum(np.abs(u[:,0]))

(1.8918058802885447, 1.428085223745682)

time: 5.64 ms


#### this ordering is very annoying.

use this rule:

 the first n columns of u and the first n rows of vt have to be kept in the same order as the singular values.

In [241]:
n = len(ss)
# reverse the n first columns of u
u[:,:n] = u[:, n-1::-1]
# reverse s
ss = ss[::-1]
# reverse the n first rows of vt
vt[:n, :] = vt[n-1::-1, :]

time: 7.83 ms


In [243]:
ss

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

time: 4.99 ms


### numpy svd

In [211]:
U, sd, Vh =  np.linalg.svd(A, full_matrices=False)

time: 79.3 ms


In [212]:
type(U), type(sd), type(Vh)

(numpy.matrixlib.defmatrix.matrix,
 numpy.ndarray,
 numpy.matrixlib.defmatrix.matrix)

time: 3.06 ms


In [213]:
U.shape, sd.shape, Vh.shape

((10000, 100), (100,), (100, 100))

time: 3.09 ms


In [249]:
largest_sd = np.argmax(sd)
largest_sd

0

time: 2.92 ms


In [231]:
sum(np.abs(U[:,largest_sd])), sum(np.abs(U[:,99]))

(matrix([[ 1.89180588]]), matrix([[ 1.54607578]]))

time: 193 ms


## sklearn

In [244]:
from sklearn.decomposition import TruncatedSVD

time: 2.91 s


In [ ]:
#from sklearn.random_projection import sparse_random_matrix
#X = sparse_random_matrix(100, 100, density=0.01, random_state=42)

In [245]:
svd = TruncatedSVD(n_components=6, random_state=42)

time: 1.14 ms


In [246]:
svd.fit(As)

TruncatedSVD(algorithm='randomized', n_components=6, n_iter=5,
       random_state=42, tol=0.0)

time: 17 ms


In [248]:
svd.components_.shape

(6, 100)

time: 2.49 ms


## Set stuff

## Summary

Faster with sparse matrices: 
    * matrix-matrix
    * matrix-vector
   
Slower:
    * vector-vector
    * scalar addition and multiplication
    * svd
   
Remember memory issues might also be a reason to pick sparse formats.